Des Funktioniert nicht, überreste von ddx versuch... wird vor dem releas gelöscht

In [7]:
import sys
import os

# Füge das Verzeichnis zum Python-Pfad hinzu
package_path = r"C:\git\Dira.ProSuiteSolution\ProSuite.Shared\py"
if package_path not in sys.path:
    sys.path.append(package_path)

# Importiere das Paket oder Modul
import prosuite.quality as quality

# Jetzt kannst du Funktionen oder Klassen aus dem Modul verwenden
DdxSpecification = quality.DdxSpecification

run_verification(quality.DdxSpecification(ddx_id=2868), output_dir, envelope)


NameError: name 'output_dir' is not defined

In [3]:
def run_verification(specification, output_dir, envelope):
    # Generate a timestamped directory for output
    global output_dir_with_timestamp  # Declare as global to use outside this function

    output_dir_with_timestamp = os.path.join(output_dir, f'output_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
    os.makedirs(output_dir_with_timestamp, exist_ok=True)
    
    # Connect to the service
    try:
        service = ps.Service(host_name='localhost', port_nr=5151)
        print(f"Connected to ProSuite service on localhost:5151")
    except Exception as e:
        print(f"Failed to connect to ProSuite service: {e}")
        return

    # Run the verification
    try:
        verification_responses = service.verify(specification=specification, output_dir=output_dir_with_timestamp, perimeter=envelope) ## Main function to run the verification
        
        issue_allowable = True  # This will control whether the process continues if non-allowable issues are found
        
        for verification_response in verification_responses:
            # Print general verification message
            print(verification_response.message)
            
            # Check for issues
            if len(verification_response.issues) > 0:
                for issue in verification_response.issues:
                    # Print issue details
                    print(issue.description)
                    print(issue.involved_objects)
                    print(issue.geometry)
                    print(issue.issue_code)
                    print(issue.allowable)
                    print(issue.stop_condition)

                    # If an issue is not allowable, stop the verification
                    if not issue.allowable:
                        print(f"Not allowed issue met: {issue.description} in {issue.involved_objects[0].table_name}")
                        print("Stopping verification")
                        issue_allowable = False
                        break

            # Break the outer loop if any non-allowable issue is found
            if not issue_allowable:
                break
            
            # Print the service call status
            print("Status: " + verification_response.service_call_status)

            # Handle different statuses
            if verification_response.service_call_status == "Failed":
                print("Server not accessible, check licence")
                print("Verification Response:" + verification_response.message)
            elif verification_response.service_call_status == "Running":
                print("Server accessible, code running")
            elif verification_response.service_call_status == "Cancelled":
                print("Verification cancelled")
            elif verification_response.service_call_status == "Finished":
                print("Verification finished successfully")
            else:
                print("Undefined status")

    except Exception as e:
        print(f"Verification failed: {e}")
    
    return output_dir_with_timestamp